In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from torch.utils.data import Dataset
import random

# ============= PARTE 1: GENERAR CARTAS SINTÉTICAS CON GAN =============
print("Paso 1: Cargando y procesando datasets...")

# Cargar los datasets
df_cleaned = pd.read_csv('clash_dataset_cleaned.csv')
df_train = pd.read_csv('clash_train.csv')
df_test = pd.read_csv('clash_test.csv')

# Preprocesamiento
numeric_cols = [
    'Cost', 'Count', 'Damage', 'Damage per second', 'Death Damage',
    'Health (+Shield)', 'Hit Speed', 'Level', 'Range', 'Spawner Health'
]
cat_cols = ['Type']

def preprocess_numeric(df):
    df_num = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    df_num = df_num.fillna(df_num.median())
    return df_num

df_num_cleaned = preprocess_numeric(df_cleaned)

# Normalización
scaler = MinMaxScaler()
df_num_cleaned_scaled = pd.DataFrame(scaler.fit_transform(df_num_cleaned), columns=numeric_cols)

# One-hot encoding
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
df_cat_cleaned = pd.DataFrame(ohe.fit_transform(df_cleaned[cat_cols]), columns=ohe.get_feature_names_out(cat_cols))

# Dataset para el modelo
df_model_cleaned = pd.concat([df_num_cleaned_scaled, df_cat_cleaned], axis=1)

print("Paso 2: Entrenando GAN...")

# GAN
latent_dim = 16
feature_dim = df_model_cleaned.shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Generator(nn.Module):
    def __init__(self, latent_dim, feature_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, feature_dim),
            nn.Sigmoid()
        )
    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(feature_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

# Entrenar GAN
G = Generator(latent_dim, feature_dim).to(device)
D = Discriminator(feature_dim).to(device)
criterion = nn.BCELoss()
optimizer_G = torch.optim.Adam(G.parameters(), lr=0.001)
optimizer_D = torch.optim.Adam(D.parameters(), lr=0.001)

real_data = torch.tensor(df_model_cleaned.values, dtype=torch.float32, device=device)

# Entrenamiento más rápido
for epoch in range(100):  # Reducido para testing
    z = torch.randn(real_data.size(0), latent_dim, device=device)
    fake_data = G(z)
    real_labels = torch.ones(real_data.size(0), 1, device=device)
    fake_labels = torch.zeros(real_data.size(0), 1, device=device)
    
    # Entrenar discriminador
    D_loss = criterion(D(real_data), real_labels) + criterion(D(fake_data.detach()), fake_labels)
    optimizer_D.zero_grad()
    D_loss.backward()
    optimizer_D.step()
    
    # Entrenar generador
    G_loss = criterion(D(fake_data), real_labels)
    optimizer_G.zero_grad()
    G_loss.backward()
    optimizer_G.step()
    
    if epoch % 25 == 0:
        print(f"Epoch {epoch}: D_loss={D_loss.item():.4f}, G_loss={G_loss.item():.4f}")

print("Paso 3: Generando cartas sintéticas...")

# GENERAR CARTAS SINTÉTICAS
z = torch.randn(10, latent_dim, device=device)
generated_structures = G(z).detach().cpu().numpy()
df_generated_structures = pd.DataFrame(generated_structures, columns=df_model_cleaned.columns)

# Desnormalizar para obtener valores reales
df_generated_numeric = pd.DataFrame(
    scaler.inverse_transform(df_generated_structures[numeric_cols]),
    columns=numeric_cols
)

# Agregar tipo de carta
df_generated_structures_final = df_generated_numeric.copy()
df_generated_structures_final['Type'] = np.random.choice(
    ['Troops and Defenses', 'Spawners', 'Damaging Spells'], 
    size=len(df_generated_structures_final)
)

print("Cartas generadas:")
print(df_generated_structures_final.head())

Paso 1: Cargando y procesando datasets...
Paso 2: Entrenando GAN...
Epoch 0: D_loss=1.3237, G_loss=0.7786
Epoch 25: D_loss=1.2036, G_loss=0.8758
Epoch 50: D_loss=1.1900, G_loss=0.8140
Epoch 75: D_loss=1.2913, G_loss=0.7070
Paso 3: Generando cartas sintéticas...
Cartas generadas:
       Cost     Count      Damage  Damage per second  Death Damage  \
0  3.805856  2.630460  131.961349         118.037445     57.870895   
1  4.683815  1.580320   77.514496          67.642181     34.646389   
2  4.067386  1.303440   64.215576          48.061951     24.379648   
3  5.011759  2.706008  139.901123         128.602997    117.304352   
4  3.854429  1.313433   45.227451          94.344208      9.707639   

   Health (+Shield)  Hit Speed     Level     Range  Spawner Health  \
0        546.599060   2.230495  7.193581  0.848657      609.560974   
1        299.628662   1.050304  7.023534  0.538788      209.534531   
2        336.953796   0.985793  7.656743  0.531862      282.545197   
3        500.894836

In [2]:
# ============= PARTE 2: CREAR DATASET SINTÉTICO PARA FINE-TUNING =============
print("\nPaso 4: Creando dataset para fine-tuning...")

def generate_clash_descriptions_from_real_cards(df_cleaned):
    """Crear descripciones basadas en las cartas reales como la Bola de Nieve"""
    
    # Plantillas inspiradas en cartas reales
    templates_by_cost = {
        (1, 2): [  # Cartas baratas
            "¡Carta económica de {cost} de elixir! {effect_description}. ¡Perfecta para ciclos rápidos!",
            "¡Solo {cost} de elixir! {effect_description}. Los enemigos no la verán venir.",
        ],
        (3, 4): [  # Cartas medias  
            "¡Esta carta de {cost} elixir es increíble! {effect_description}. ¡Úsala sabiamente!",
            "¡Carta versátil de {cost} elixir! {effect_description}. Cambiará el rumbo de la batalla.",
        ],
        (5, 6): [  # Cartas caras
            "¡Carta poderosa de {cost} elixir! {effect_description}. ¡Devastación garantizada!",
            "¡Esta carta de {cost} elixir es formidable! {effect_description}. Los enemigos temerán su poder.",
        ],
        (7, 10): [  # Cartas Legendarias
            "¡Carta legendaria de {cost} elixir! {effect_description}. ¡Dominará toda la arena!",
            "¡Increíble carta de {cost} elixir! {effect_description}. ¡La victoria está asegurada!",
        ]
    }
    
    synthetic_data = []
    
    for _, row in df_cleaned.iterrows():
        cost = int(row.get('Cost', 3))
        damage = int(row.get('Damage', 0))
        health = int(row.get('Health (+Shield)', 0))
        card_type = row.get('Type', 'Troops and Defenses')
        
        # Seleccionar template por costo
        template_key = None
        for cost_range, templates in templates_by_cost.items():
            if cost_range[0] <= cost <= cost_range[1]:
                template_key = cost_range
                break
        
        if not template_key:
            template_key = (3, 4)
            
        template = random.choice(templates_by_cost[template_key])
        
        # Generar descripción del efecto basada en stats
        if card_type == 'Damaging Spells':
            if damage > 800:
                effect = f"Causa {damage} de daño devastador en área"
            elif damage > 400:
                effect = f"Inflige {damage} de daño en zona amplia"
            else:
                effect = f"Daña enemigos con {damage} puntos"
        elif card_type == 'Spawners':
            effect = f"Genera tropas continuamente con gran resistencia"
        else:  # Troops and Defenses
            if health > 2000:
                effect = f"Tanque resistente con {health} de vida que aguanta mucho daño"
            elif damage > 400:
                effect = f"Guerrero feroz que causa {damage} de daño por golpe"  
            else:
                effect = f"Unidad equilibrada con {damage} de ataque y {health} de vida"
        
        description = template.format(cost=cost, effect_description=effect)
        
        synthetic_data.append({
            'Card': row.get('Card', 'Carta Nueva'),
            'Cost': cost,
            'Type': card_type,
            'Description': description,
            'Attributes': f"Coste: {cost}, Daño: {damage}, Salud: {health}, Tipo: {card_type}"
        })
    
    return pd.DataFrame(synthetic_data)

# Generar dataset sintético
synthetic_descriptions = generate_clash_descriptions_from_real_cards(df_cleaned)
print("Dataset sintético creado:")
print(synthetic_descriptions.head(3))


Paso 4: Creando dataset para fine-tuning...
Dataset sintético creado:
          Card  Cost                 Type  \
0      Archers     3  Troops and Defenses   
1  Baby Dragon     4  Troops and Defenses   
2      Balloon     5  Troops and Defenses   

                                         Description  \
0  ¡Carta versátil de 3 elixir! Unidad equilibrad...   
1  ¡Esta carta de 4 elixir es increíble! Unidad e...   
2  ¡Esta carta de 5 elixir es formidable! Guerrer...   

                                          Attributes  
0  Coste: 3, Daño: 86, Salud: 254, Tipo: Troops a...  
1  Coste: 4, Daño: 133, Salud: 1064, Tipo: Troops...  
2  Coste: 5, Daño: 798, Salud: 1396, Tipo: Troops...  


In [3]:
# ============= PARTE 3: FINE-TUNING =============
print("\nPaso 5: Preparando fine-tuning...")

class ClashRoyaleDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = []
        
        for _, row in df.iterrows():
            prompt = f"[CLASH] {row['Attributes']} -> "
            target = f"{row['Description']}<|endoftext|>"
            full_text = prompt + target
            self.data.append(full_text)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': encoding['input_ids'].flatten()
        }

# Cargar modelo
print("Cargando GPT-2...")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Configurar tokens
tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
model.resize_token_embeddings(len(tokenizer))

# Dataset
train_dataset = ClashRoyaleDataset(synthetic_descriptions, tokenizer)

# Configuración de entrenamiento
training_args = TrainingArguments(
    output_dir='./clash-royale-model',
    overwrite_output_dir=True,
    num_train_epochs=3,  # Reducido para testing
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=50,
    logging_steps=25,
    save_steps=200,
    eval_strategy='no',
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    remove_unused_columns=False,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

print("Iniciando fine-tuning...")
trainer.train()

# Guardar modelo
model.save_pretrained('./clash-royale-model')
tokenizer.save_pretrained('./clash-royale-model')


Paso 5: Preparando fine-tuning...
Cargando GPT-2...


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
C:\Users\briam\AppData\Local\Temp\ipykernel_14460\2923818107.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Iniciando fine-tuning...


Step,Training Loss
25,8.241300
50,6.115200
75,4.226200
100,3.357900


('./clash-royale-model\\tokenizer_config.json',
 './clash-royale-model\\special_tokens_map.json',
 './clash-royale-model\\vocab.json',
 './clash-royale-model\\merges.txt',
 './clash-royale-model\\added_tokens.json')

In [4]:
########## GENERAR NARRATIVAS =============
import pandas as pd
import numpy as np
import random
import re
import unicodedata

print("Sistema de narrativas en español...")

def clean_spanish_text_advanced(text):
    """Limpieza avanzada para garantizar español puro"""
    
    # Eliminar palabras problemáticas en inglés
    english_words = [
        'vernal', 'recommended', 'spawner', 'locations', 'troops', 'defenses', 
        'damage', 'health', 'attack', 'defense', 'spell', 'building', 'unit'
    ]
    
    for word in english_words:
        text = re.sub(r'\b' + word + r'\b', '', text, flags=re.IGNORECASE)
    
    # Eliminar caracteres problemáticos
    text = re.sub(r'[^\w\s\.\!\?\,\:\;áéíóúÁÉÍÓÚñÑüÜ]', '', text)
    text = re.sub(r'\?+', '', text)  # Eliminar signos de interrogación múltiples
    text = re.sub(r'\s+', ' ', text)  # Normalizar espacios
    
    return text.strip()

def generate_premium_clash_narrative(attributes):
    """Generador premium de narrativas 100% en español auténtico"""
    
    cost = int(attributes.get('Cost', 3))
    damage = int(attributes.get('Damage', 0))  
    health = int(attributes.get('Health (+Shield)', 0))
    card_type = attributes.get('Type', 'Troops and Defenses')
    
    # Frases de impacto auténticas de Clash Royale
    exclamations = [
        "¡Esta carta es increíble!",
        "¡Una carta formidable!",
        "¡Carta devastadora!",
        "¡Poder absoluto!",
        "¡Fuerza imparable!"
    ]
    
    # Plantillas por tipo y características específicas
    if card_type == 'Damaging Spells':
        if cost <= 2:  # Hechizos baratos
            templates = [
                f"¡Hechizo rápido de {cost} elixir! Causa {damage} de daño instantáneo. Perfecto para eliminar tropas pequeñas y sorprender al enemigo.",
                f"¡Magia económica de {cost} elixir! Inflige {damage} puntos de daño en área. Los rivales no verán venir este devastador conjuro.",
                f"¡Conjuro veloz de {cost} elixir! Destruye con {damage} de daño. Ideal para ciclos rápidos y ataques sorpresa definitivos."
            ]
        elif cost <= 4:  # Hechizos medios
            templates = [
                f"¡Hechizo poderoso de {cost} elixir! Arrasa enemigos con {damage} de daño brutal en zona amplia. Cambiará el destino de la batalla.",
                f"¡Conjuro versátil de {cost} elixir! Aniquila rivales con {damage} puntos de daño devastador. Úsalo sabiamente para la victoria.",
                f"¡Magia destructiva de {cost} elixir! Causa {damage} de daño letal en área. Los enemigos huirán aterrorizados del campo."
            ]
        else:  # Hechizos caros
            templates = [
                f"¡Hechizo legendario de {cost} elixir! Aniquila todo con {damage} de daño masivo apocalíptico. Devastación total garantizada en la arena.",
                f"¡Conjuro supremo de {cost} elixir! Destrucción absoluta de {damage} puntos letales. Dominará completamente toda la arena de batalla.",
                f"¡Magia definitiva de {cost} elixir! Poder destructivo de {damage}. La victoria está completamente asegurada para siempre."
            ]
            
    elif card_type == 'Spawners':
        if cost <= 4:  # Estructuras baratas
            templates = [
                f"¡Torre productora de {cost} elixir! Genera tropas continuamente sin parar. Resistencia sólida para defender tu corona victoriosamente.",
                f"¡Edificio spawner de {cost} elixir! Invoca unidades automáticamente en oleadas. Presión constante que abrumará a los enemigos.",
                f"¡Estructura generadora de {cost} elixir! Produce ejércitos sin descanso. Los rivales no podrán avanzar ni un solo paso."
            ]
        else:  # Estructuras caras
            templates = [
                f"¡Mega fortaleza de {cost} elixir! Genera oleadas masivas de tropas imparables. Dominará completamente el campo de batalla enemigo.",
                f"¡Super edificio de {cost} elixir! Invoca ejércitos legendarios continuamente. La presión será absolutamente abrumadora para los rivales.",
                f"¡Torre suprema de {cost} elixir! Producción masiva garantizada eternamente. Los enemigos se rendirán antes de la primera oleada."
            ]
            
    else:  # Troops and Defenses
        if cost <= 2:  # Tropas baratas
            if damage > 100:
                templates = [
                    f"¡Guerrero feroz de {cost} elixir! Ataque brutal de {damage} por golpe mortal. Perfecto para ataques sorpresa devastadores.",
                    f"¡Luchador veloz de {cost} elixir! Golpea con {damage} de daño letal. Los enemigos caerán antes de reaccionar.",
                    f"¡Asesino rápido de {cost} elixir! Causa {damage} puntos por impacto. Ideal para ciclos de muerte imparables."
                ]
            else:
                templates = [
                    f"¡Tropa económica de {cost} elixir! Resistencia sólida de {health} puntos. Perfecta para distraer y confundir enemigos.",
                    f"¡Unidad barata de {cost} elixir! Aguanta {health} de daño heroicamente. Excelente para defensa y contraataques.",
                    f"¡Soldado accesible de {cost} elixir! Vida resistente de {health}. Los rivales gastarán elixir extra innecesariamente."
                ]
                
        elif cost <= 4:  # Tropas medias
            if damage > 300:
                templates = [
                    f"¡Guerrero implacable de {cost} elixir! Daño devastador de {damage} por golpe brutal. Arrasará con cualquier enemigo del camino.",
                    f"¡Luchador legendario de {cost} elixir! Ataque mortal de {damage} puntos. Los rivales huirán aterrorizados de su poder.",
                    f"¡Soldado feroz de {cost} elixir! Golpe letal de {damage}. Devastación pura que aniquilará toda resistencia enemiga."
                ]
            elif health > 1500:
                templates = [
                    f"¡Tanque invencible de {cost} elixir! Vida masiva de {health} puntos épicos. Absorbe todo el daño enemigo sin inmutarse.",
                    f"¡Muro viviente de {cost} elixir! Resistencia titanica de {health}. Ningún ataque enemigo podrá detener su avance.",
                    f"¡Fortaleza móvil de {cost} elixir! Aguanta {health} de daño heroicamente. Los enemigos se cansarán de atacar inútilmente."
                ]
            else:
                templates = [
                    f"¡Tropa equilibrada de {cost} elixir! Combina {damage} de ataque y {health} de vida perfectamente. Versatilidad total asegurada.",
                    f"¡Unidad completa de {cost} elixir! Estadísticas balanceadas ideales para toda situación. Funcionará en cualquier estrategia.",
                    f"¡Soldado versátil de {cost} elixir! Poder y resistencia combinados magistralmente. Perfecto para cualquier táctica de batalla."
                ]
                
        else:  # Tropas caras (5+ elixir)
            if damage > 500:
                templates = [
                    f"¡Bestia legendaria de {cost} elixir! Poder destructivo de {damage} apocalíptico. Aniquilará completamente cualquier ejército enemigo existente.",
                    f"¡Titán imparable de {cost} elixir! Fuerza brutal de {damage} devastadora. Los rivales abandonarán la partida al verlo aparecer.",
                    f"¡Monstruo definitivo de {cost} elixir! Daño letal de {damage}. Dominación total asegurada para toda la eternidad."
                ]
            elif health > 3000:
                templates = [
                    f"¡Coloso invencible de {cost} elixir! Resistencia épica de {health} puntos legendarios. Será completamente imposible de destruir.",
                    f"¡Gigante supremo de {cost} elixir! Vida masiva de {health}. Absorbe cualquier ataque sin sufrir daño significativo.",
                    f"¡Titán defensivo de {cost} elixir! Aguanta {health} de daño épico. Los enemigos se agotarán antes de derrotarlo."
                ]
            else:
                templates = [
                    f"¡Campeón premium de {cost} elixir! Estadísticas superiores balanceadas perfectamente. Dominará toda la arena con superioridad.",
                    f"¡Unidad élite de {cost} elixir! Poder y resistencia combinados magistralmente. La victoria está completamente garantizada.",
                    f"¡Guerrero supremo de {cost} elixir! Perfección absoluta en combate. Los rivales no tienen ni la menor oportunidad."
                ]
    
    # Seleccionar template aleatorio y limpiarlo
    selected_template = random.choice(templates)
    clean_narrative = clean_spanish_text_advanced(selected_template)
    
    # Validación final
    if len(clean_narrative.split()) < 8:
        fallback = f"¡Carta poderosa de {cost} elixir! Perfecta para dominar la arena. Los enemigos temerán su increíble poder destructivo."
        return clean_spanish_text_advanced(fallback)
    
    return clean_narrative

def validate_spanish_quality(narratives):
    """Valida que las narrativas sean 100% español"""
    
    quality_report = {
        'spanish_purity': 0,
        'appropriate_length': 0, 
        'clash_authenticity': 0,
        'exclamation_usage': 0,
        'total_quality': 0
    }
    
    clash_keywords = ['elixir', 'daño', 'carta', 'enemigo', 'ataque', 'batalla', 'arena', 'poder', 'tropa']
    english_indicators = ['vernal', 'recommended', 'spawner', 'damage', 'health', '?']
    
    for narrative in narratives:
        score = 0
        
        # Narrativas en Español
        has_english = any(word in narrative.lower() for word in english_indicators)
        if not has_english:
            quality_report['spanish_purity'] += 1
            score += 1
        
        # Longitud apropiada (15-40 palabras)
        word_count = len(narrative.split())
        if 15 <= word_count <= 40:
            quality_report['appropriate_length'] += 1
            score += 1
        
        # Autenticidad Clash Royale
        has_clash_words = any(keyword in narrative.lower() for keyword in clash_keywords)
        if has_clash_words:
            quality_report['clash_authenticity'] += 1
            score += 1
        
        # Uso de exclamaciones
        if '¡' in narrative and '!' in narrative:
            quality_report['exclamation_usage'] += 1
            score += 1
        
        quality_report['total_quality'] += score / 4
    
    # Calcular promedios
    total_narratives = len(narratives)
    for key in quality_report:
        if key != 'total_quality':
            quality_report[key] = (quality_report[key] / total_narratives) * 100
        else:
            quality_report[key] = (quality_report[key] / total_narratives) * 100
    
    return quality_report

# Verificar datos de cartas
if 'df_generated_structures_final' not in locals():
    print("Creando cartas de ejemplo para testing...")
    np.random.seed(42)
    df_generated_structures_final = pd.DataFrame({
        'Cost': np.random.randint(1, 9, 8),
        'Damage': np.random.randint(50, 800, 8),
        'Health (+Shield)': np.random.randint(100, 3000, 8),
        'Type': np.random.choice(['Troops and Defenses', 'Spawners', 'Damaging Spells'], 8)
    })

print("Generando narrativas en español...")

# GENERAR NARRATIVAS 
premium_narratives = []

for i, (_, row) in enumerate(df_generated_structures_final.iterrows()):
    narrative = generate_premium_clash_narrative(row.to_dict())
    premium_narratives.append(narrative)
    
    print(f"\n=== CARTA #{i+1} ===")
    print(f"Coste: {row['Cost']:.0f} elixir")
    print(f"Daño: {row['Damage']:.0f}")
    print(f"Salud: {row['Health (+Shield)']:.0f}")
    print(f"Tipo: {row['Type']}")
    print(f"Narrativa: {narrative}")
    print("-" * 70)

print(f"\nNARRATIVAS COMPLETADAS!")
print(f"Total generado: {len(premium_narratives)}")

# VALIDACIÓN DE CALIDAD AVANZADA
print("\nREPORTE DE CALIDAD AVANZADO:")
quality_metrics = validate_spanish_quality(premium_narratives)

for metric, score in quality_metrics.items():
    status = "✅ EXCELENTE" if score >= 90 else "⚠️ BUENO" if score >= 70 else "❌ MEJORABLE"
    print(f"{metric.replace('_', ' ').title()}: {score:.1f}% {status}")

# Verificar que no hay palabras problemáticas
print("\nVERIFICACIÓN DE LAS NARRATIVAS:")
problematic_words = []
for i, narrative in enumerate(premium_narratives):
    if any(word in narrative.lower() for word in ['vernal', '?', 'recommended', 'spawner']):
        problematic_words.append(f"Carta {i+1}: {narrative}")

if problematic_words:
    print("NARRATIVAS CON PROBLEMAS:")
    for problem in problematic_words:
        print(f"  - {problem}")
else:
    print("TODAS LAS NARRATIVAS GENERADAS ESTÁN APROPIADAS!")

# Guardar resultados
results_premium = pd.DataFrame({
    'Cost': [row['Cost'] for _, row in df_generated_structures_final.iterrows()],
    'Damage': [row['Damage'] for _, row in df_generated_structures_final.iterrows()],
    'Health': [row['Health (+Shield)'] for _, row in df_generated_structures_final.iterrows()],
    'Type': [row['Type'] for _, row in df_generated_structures_final.iterrows()],
    'Narrative': premium_narratives
})

results_premium.to_csv('clash_cards_narratives.csv', index=False, encoding='utf-8')
print("\nResultados guardados en 'clash_cards_narratives.csv'")

# Mostrar ejemplos finales
print("\nEJEMPLOS DE NARRATIVAS: ")
for i, (_, row) in enumerate(results_premium.head(3).iterrows()):
    print(f"{i+1}. Coste {row['Cost']:.0f}: \"{row['Narrative']}\"")

Sistema de narrativas en español...
Generando narrativas en español...

=== CARTA #1 ===
Coste: 4 elixir
Daño: 132
Salud: 547
Tipo: Troops and Defenses
Narrativa: Soldado versátil de 3 elixir! Poder y resistencia combinados magistralmente. Perfecto para cualquier táctica de batalla.
----------------------------------------------------------------------

=== CARTA #2 ===
Coste: 5 elixir
Daño: 78
Salud: 300
Tipo: Spawners
Narrativa: Torre productora de 4 elixir! Genera tropas continuamente sin parar. Resistencia sólida para defender tu corona victoriosamente.
----------------------------------------------------------------------

=== CARTA #3 ===
Coste: 4 elixir
Daño: 64
Salud: 337
Tipo: Damaging Spells
Narrativa: Hechizo poderoso de 4 elixir! Arrasa enemigos con 64 de daño brutal en zona amplia. Cambiará el destino de la batalla.
----------------------------------------------------------------------

=== CARTA #4 ===
Coste: 5 elixir
Daño: 140
Salud: 501
Tipo: Damaging Spells
Narrativa: 